In [ ]:
import os
import re
import math
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn import Linear, ReLU, Dropout, Conv2d, MaxPool2d
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.optim import AdamW

import gurobipy as gb
from gurobipy import GRB
import time

### Check if GPU available

In [ ]:
# set CUDA_VISIBLE_DEVICES=0
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
train_test_dir = os.path.join(os.getcwd(), "dataGeneration/preprocessed_data_test_nopadding")

with open(os.path.join(train_test_dir, f"X_test.pkl"), 'rb') as f:
    X_test = pickle.load(f)
    f.close()

with open(os.path.join(train_test_dir, f"y_test.pkl"), 'rb') as f:
    y_test = pickle.load(f)
    f.close()

with open(os.path.join(train_test_dir, f"indices_test.pkl"), 'rb') as f:
    index_test = pickle.load(f)
    f.close()

with open(os.path.join(train_test_dir, f"schedule_test.pkl"), 'rb') as f:
    schedule_test = pickle.load(f)
    f.close()

solTime_test = np.load(os.path.join(train_test_dir, "solTime_test.npy"))
objVal_test = np.load(os.path.join(train_test_dir, "objVal_test.npy"))
model_test = np.load(os.path.join(train_test_dir, "model_test.npy")).astype("int32")


In [ ]:
print((X_test[0].shape))
print(len(y_test))
print(len(index_test))

print(solTime_test.shape)
print(objVal_test.shape)
print(model_test.shape)


### Building the Transformer network

In [ ]:
# timestep = X_test[0].shape[1]
dimensions = X_test[0].shape[2] # feature size

nbTime, nbBus, nbSolar, nbScen = 48, 33, 3, X_test[0].shape[0]

charging_station = np.squeeze(pd.read_csv(os.path.join(os.path.join(os.getcwd(), 'systemData'), 'cs_params_variable.csv')).to_numpy())
nbCS = len(charging_station)

data_dir = os.path.join(os.getcwd(), 'systemData')
EV_routes = pd.read_csv(os.path.join(data_dir, 'EV_routes.csv')).to_numpy()
nbRoute = EV_routes.shape[0]

nbOut = (nbRoute*(nbTime-1) + nbCS*nbTime*2)

print(dimensions, nbOut)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.transpose(0,1))

    def forward(self, x):
        x = x + self.pe[:,:x.size(1),:]
        return self.dropout(x)

class Transformer(nn.Module):
    def __init__(self, dimensions: int, out_dim: int, d_model=256, nhead=4, num_layers=2, dim_feedforward=512, dropout=0.1, ffn=256):
        super(Transformer, self).__init__()

        self.d_model = 64
        self.dim_feedforward = 256
        self.nhead = 4
        self.dp = 0.3
        self.ffn = 256

        self.embedding = nn.Linear(dimensions, self.d_model)

        # self.pos_encoder = PositionalEncoding(self.d_model)

        transformer_layer = nn.TransformerEncoderLayer(d_model=self.d_model, nhead=self.nhead, dim_feedforward=self.dim_feedforward, dropout=self.dp, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(transformer_layer, num_layers=num_layers)

        self.classifier = nn.Sequential(
            # nn.MaxPool1d(self.d_model),
            # nn.Flatten(),
            nn.Linear(self.d_model, self.ffn),
            nn.ReLU(),
            nn.Linear(self.ffn, self.ffn),
            nn.ReLU(),
            nn.Linear(self.ffn, out_dim),
            nn.Sigmoid())
        
        
    def forward(self, x):
        x = self.embedding(x)
        # x = self.pos_encoder(x)
        # print(x.shape)
        x = self.transformer_encoder(x)
        x = x[:,(nbBus*2+nbSolar):, :]

        x = self.classifier(x)
        return x
        

### Create Dataset and DataLoader

In [ ]:
config = {
        'batch_size' : 1, # Num samples to average over for gradient updates
        'EPOCHS' : 200, # Num times to iterate over the entire dataset
        'LEARNING_RATE' : 5e-4, # Learning rate for the optimizer
        'WEIGHT_DECAY' : 1e-3, # Weight decay parameter for the Adam optimizer
    }

In [ ]:
class coordinationDataset(TensorDataset):
    def __init__(self, X, y):
        super(coordinationDataset, self).__init__()
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        X = self.X[index]
        y = self.y[index]
        
        X_tensor = torch.tensor(X, dtype=torch.float32)
        y_tensor = torch.round(torch.tensor(y, dtype=torch.float32))

        return X_tensor, y_tensor
    
    def __len__(self):
        return len(self.X)

In [ ]:
test_dataset = coordinationDataset(X_test, y_test)


In [ ]:
net = Transformer(dimensions=dimensions, out_dim=nbOut)

batch_size = config['batch_size']
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

optimizer = optim.Adam(net.parameters(), lr=config["LEARNING_RATE"])


#### Define custom loss function

In [ ]:
def asymmetric_loss(predict, target, gamma_neg=0.3, gamma_pos=0, clip=0.0, eps=1e-8, disable_torch_grad_focal_loss=True):

    """"
    Parameters
    ----------
    x: input logits
    y: targets (multi-label binarized vector)
    """

    # Calculating Probabilities
    x_sigmoid = predict
    xs_pos = x_sigmoid
    xs_neg = 1 - x_sigmoid

    # Asymmetric Clipping
    if clip is not None and clip > 0:
        xs_neg = (xs_neg + clip).clamp(max=1)

    # Basic CE calculation
    los_pos = target * torch.log(xs_pos.clamp(min=eps))
    los_neg = (1 - target) * torch.log(xs_neg.clamp(min=eps))
    loss = los_pos + los_neg

    # Asymmetric Focusing
    if gamma_neg > 0 or gamma_pos > 0:
        if disable_torch_grad_focal_loss:
            torch.set_grad_enabled(False)
        pt0 = xs_pos * target
        pt1 = xs_neg * (1 - target)  # pt = p if t > 0 else 1-p
        pt = pt0 + pt1
        one_sided_gamma = gamma_pos * target + gamma_neg * (1 - target)
        one_sided_w = torch.pow(1 - pt, one_sided_gamma)
        if disable_torch_grad_focal_loss:
            torch.set_grad_enabled(True)
        loss *= one_sided_w

    return -loss.sum()


### Model Testing

In [ ]:
# load the model
interval = 15

net = Transformer(dimensions=dimensions, out_dim=nbOut)
net.load_state_dict(torch.load(os.path.join(os.getcwd(), f"ML_Model/transformer_coordination_nopadding_{interval}.pth")))

In [ ]:
# test number of feasible solutions
# test the model on the test set
net.eval()
net.to(device)

#### Testing of bit accuracy

In [ ]:
thres = 0.5

one_accuracy = []
zero_accuracy = []
bit_accuracy = []
running_loss = 0
mean_one = []
mean_zero = []

for j, data in enumerate(test_loader):
    
    net.eval()
    inputs_all, labels_all = data

    # do a for loop to perform perdiction for each scenario
    output_append = torch.tensor([]).to(device) 
    gt_append = torch.tensor([]).to(device) 

    for x in range(nbScen):
        
        inputs, labels = inputs_all[:,x,:,:].to(device), labels_all[:,x,:,].to(device)       
        optimizer.zero_grad()
        outputs = net(inputs)

        outputs = outputs.flatten().reshape(labels.shape[0],-1,)

        output_append = torch.concat((output_append, outputs),dim=1)
        gt_append = torch.concat((gt_append, labels),dim=1)

    output_append = output_append.reshape(-1,)
    gt_append = gt_append.reshape(-1,)

    # loss_fn = nn.BCELoss()
    # loss = loss_fn(outputs, labels)
    # loss = asymmetric_loss(outputs, labels)
    # running_loss += loss.item()

    # start testing
    outputs = (output_append).reshape(-1,)   
    outputs_percent = outputs
    outputs = torch.where(outputs >= thres, torch.ceil(outputs), torch.floor(outputs)).reshape(-1,)
    # outputs = torch.round(outputs)
    labels = gt_append.reshape(-1,)

    one_labels = torch.where(labels == 1)
    zero_labels = torch.where(labels == 0)
    
    one_outputs = outputs[one_labels]
    zero_outputs = outputs[zero_labels]

    one_acc = 1 - torch.sum(torch.abs(1 - one_outputs)) / one_outputs.shape[0] # 1 minus percentage of error
    zero_acc = 1 - torch.sum(torch.abs(0 - zero_outputs)) / zero_outputs.shape[0]
    bit_acc = 1 - torch.sum(torch.abs(outputs - labels)) / labels.shape[0]

    one_accuracy.append(one_acc.cpu().detach().numpy())
    zero_accuracy.append(zero_acc.cpu().detach().numpy())
    bit_accuracy.append(bit_acc.cpu().detach().numpy())

    # mean acc
    id_1 = torch.where(outputs == 1)
    id_0 = torch.where(outputs == 0)

    p_1 = outputs_percent[id_1]
    p_0 = outputs_percent[id_0]


    y_1 = labels[id_1]
    y_0 = labels[id_0]

    y_1_1 = torch.where(y_1 == 1)
    y_1_0 = torch.where(y_1 == 0)
    y_0_1 = torch.where(y_0 == 1)
    y_0_0 = torch.where(y_0 == 0)

    avg_1 = torch.mean(torch.cat((p_1[y_1_1], torch.ones(y_1_0[0].shape[0]).to(device) - p_1[y_1_0])))
    avg_0 = torch.mean(torch.cat((p_0[y_0_1], torch.ones(y_0_0[0].shape[0]).to(device) - p_0[y_0_0])))

    # avg_1 = torch.mean(torch.cat((p_1[y_1_1],  p_1[y_1_0])))
    # avg_0 = torch.mean(torch.cat((p_0[y_0_1], p_0[y_0_0])))

    # avg_1 = torch.mean(p_1[y_1_1])
    # avg_0 = torch.mean(p_0[y_0_1])

    # avg_1 = torch.mean(p_1[y_1_0])
    # avg_0 = torch.mean(p_0[y_0_1])

    mean_one.append(avg_1.cpu().detach().numpy())
    mean_zero.append(avg_0.cpu().detach().numpy())

print("Average one bit accuracy", np.mean(one_accuracy))
print("Average zero bit accuracy", np.mean(zero_accuracy))
print("Average bit accuracy", np.mean(bit_accuracy))
print('Loss:', running_loss / len(test_loader))
print(np.mean(mean_one), np.mean(mean_zero))

### Test for baseline solving speed (speed cap at 300 seconds)

In [ ]:
# read in paramaeters
data_dir = os.path.join(os.getcwd(), 'systemData')
EV_routes = pd.read_csv(os.path.join(data_dir, 'EV_routes.csv')).to_numpy()
bus_params = pd.read_csv(os.path.join(data_dir, 'bus_params.csv')).to_numpy()
# EV_schedules = pd.read_csv(os.path.join(data_dir, 'EV_schedules.csv')).to_numpy().astype("int32")

nbScen, nbTime, nbBus, nbRoute = 5, 48, bus_params.shape[0], EV_routes.shape[0]
traffic = np.zeros(nbTime-1)
traffic[14:20] = 1      # from 7-10am 
traffic[32:40] = 1      # from 4-8pm

charging_station = np.squeeze(pd.read_csv(os.path.join(data_dir, 'cs_params_variable.csv')).to_numpy())
non_charging_station = np.array([i for i in range(nbBus) if i not in charging_station])
nbCS = len(charging_station)

normal_nodes =  list(charging_station) + list(range(101,108))
virtual_nodes = list(range(201,205))
congest_nodes = list(range(301,324))

always_arc, normal_arc, congest_arc = [], [], []

for r in range(nbRoute):
    if (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in (normal_nodes+virtual_nodes)) and (EV_routes[r,1] != EV_routes[r,2]):
        normal_arc.append(r)
    elif (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in congest_nodes):
        congest_arc.append(r)
    else:
        always_arc.append(r)


In [ ]:
gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)
    
# loop through all test models and calculate average optimization time
opt_time = []
opt_val = []
opt_ones = []
for i, _ in enumerate(model_test):
    
    runtime = solTime_test[i]
    obj = objVal_test[i]
    
    print("Optimization time for model ", i, ": ", runtime)
    print("Optimization Value for model ", i, ": ", obj)
    
    opt_time.append(runtime)
    opt_val.append(obj)

    nbEV = np.max(schedule_test[i][:,0]) + 1
    binary_vars = y_test[i][0:(nbRoute*(nbTime-1) + nbCS*nbTime*2)*nbEV]

    opt_ones.append(np.count_nonzero(np.round(binary_vars)))


In [ ]:
opt_dict = {
    "opt_time": opt_time,
    "opt_val": opt_val,
    "opt_ones": opt_ones
}

result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, "opt_test_nopadding.pkl"), 'wb') as f:
    pickle.dump(opt_dict, f)

In [ ]:
result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, "opt_test_nopadding.pkl"), 'rb') as f:
    opt_dict = pickle.load(f)

opt_time = opt_dict["opt_time"]
opt_val = opt_dict["opt_val"]
opt_ones = opt_dict["opt_ones"]

In [ ]:
# flatten opt_time
print("Average optimization time: ", np.mean(opt_time))
opt_time_baseline = np.mean(opt_time)
# flatten opt_time
print("Average optimization value: ", np.mean(opt_val))
opt_val_baseline = np.mean(opt_val)
print("Number of ones: ", opt_ones)

### Calculate the optimization time for each instance in the test set if we use $\color{lightblue}\text{equality constraint}$ to find optimal solution 

#### Start testing equality constraint

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)

# loop through all test models and calculate average optimization time
opt_time_thres = []
opt_val_thres = []
opt_ones_thres = []
opt_utilized_thres = []
for i, x in enumerate(model_test):
    
    path = os.path.join(os.getcwd(), "dataGeneration/model_test")

    model = gb.read(os.path.join(path, f"coordination_{x}.mps"), env=gurobi_env)
    model.setParam("OutputFlag", 0)
    model.setParam("TimeLimit", 5*60)

    ################# timing the first code block #####################
    start = time.time()

    # deep learning prediciton
    # do a for loop to perform perdiction for each scenario
    output_append = torch.tensor([]).to(device) 

    for s in range(nbScen):
        inputs = torch.tensor(np.expand_dims(test_dataset.X[i][s], axis=0), dtype=torch.float32) 
        inputs = inputs.to(device)    
        outputs = net(inputs) 

        output_append = torch.concat((output_append, outputs),dim=1)

    output_append = output_append.reshape(-1,)

    ################## end #########################

    y_pred_binary =  (output_append).reshape(-1,).cpu().detach().numpy()
    # y_pred_binary = y_test[i].flatten()
    nbEV = int(y_test[i].shape[1]/nbOut)
    print(nbEV)

    modelVars = model.getVars()

    one_threshold = (np.mean(mean_one))
    zero_threshold = (1-np.mean(mean_zero))

    bin_id = []
    ###### Build the index for the variables ######
    for sc in range(nbScen):
        for k in range(nbEV):
            # for each EV get each attribute and order it based on EV number
            for r in range(nbRoute):
                for t in range(nbTime-1):
                    var = model.getVarByName(f"EVArcStatus[{sc},{k},{r},{t}]")

                    bin_id.append(var.index)

            for c in range(nbCS):
                for t in range(nbTime):
                    var = model.getVarByName(f"EVChargeStatus[{sc},{k},{c},{t}]")

                    bin_id.append(var.index)
                    
            for c in range(nbCS):
                for t in range(nbTime):
                    var = model.getVarByName(f"EVDischargeStatus[{sc},{k},{c},{t}]")

                    bin_id.append(var.index)

    # use equality constestt from here on
    for j in range(len(y_pred_binary)):
        if (y_pred_binary[j] >= one_threshold or y_pred_binary[j] <= zero_threshold):
            modelVars[bin_id[j]].setAttr("LB", round(y_pred_binary[j]))
            modelVars[bin_id[j]].setAttr("UB", round(y_pred_binary[j]))
            
    end = time.time()
    runtime1 = end - start

    opt_ones_thres.append(np.count_nonzero(y_pred_binary >= one_threshold))
    opt_utilized_thres.append(np.count_nonzero(y_pred_binary >= one_threshold)+  np.count_nonzero(y_pred_binary <= zero_threshold))

    ################# timing the second code block #####################
    start = time.time()

    model.optimize()

    end = time.time()
    runtime2 = end - start
    runtime = runtime1 + runtime2
    ################## end #########################
    
    try:
        print("Optimization time for model ", i, ": ", model.ObjVal)
        print("Optimization value for model ", i, ": ", runtime)
        opt_time_thres.append(runtime)
        opt_val_thres.append(model.ObjVal)
    except:
        print("infeasible")

        # model.computeIIS()

        # for c in model.getConstrs():
        #     if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')

        # for v in model.getVars():
        #     if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
        #     if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')
    
    model.dispose()


In [ ]:
opt_dict_thres = {
    "opt_time": opt_time_thres,
    "opt_val": opt_val_thres,
    "opt_ones": opt_ones_thres,
    "opt_utilized": opt_utilized_thres
}

result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, "transformer_opt_thres_test_nopadding.pkl"), 'wb') as f:
    pickle.dump(opt_dict_thres, f)

In [ ]:
with open(os.path.join(result_path, "transformer_opt_thres_test_nopadding.pkl"), 'rb') as f:
    opt_dict_thres = pickle.load(f)

opt_time_thres = opt_dict_thres["opt_time"]
opt_val_thres = opt_dict_thres["opt_val"]
opt_ones_thres = opt_dict_thres["opt_ones"]
opt_utilized_thres = opt_dict_thres["opt_utilized"]

In [ ]:
# flatten opt_time
opt_time_thres_mean = np.mean(opt_time_thres)
print("Average optimization time: ", opt_time_thres_mean)
# flatten opt_time
opt_val_thres_mean = np.mean(opt_val_thres)
print("Average optimization value: ", np.mean(opt_val_thres))
print("Number of feasible model", len(opt_time_thres))
print("Number of ones: ", opt_ones_thres)
print("Number of variable utilized ", opt_utilized_thres)

### Testing equality constraint with feasibility check

In [ ]:
# read in paramaeters
data_dir = os.path.join(os.getcwd(), 'systemData')
EV_routes = pd.read_csv(os.path.join(data_dir, 'EV_routes.csv')).to_numpy()
bus_params = pd.read_csv(os.path.join(data_dir, 'bus_params.csv')).to_numpy()
# EV_schedules = pd.read_csv(os.path.join(data_dir, 'EV_schedules.csv')).to_numpy().astype("int32")

nbTime, nbBus, nbRoute = 48, bus_params.shape[0], EV_routes.shape[0]
traffic = np.zeros(nbTime-1)
traffic[14:20] = 1      # from 7-10am 
traffic[32:40] = 1      # from 4-8pm

charging_station = np.squeeze(pd.read_csv(os.path.join(data_dir, 'cs_params_variable.csv')).to_numpy())
non_charging_station = np.array([i for i in range(nbBus) if i not in charging_station])
nbCS = len(charging_station)

normal_nodes =  list(charging_station) + list(range(101,108))
virtual_nodes = list(range(201,205))
congest_nodes = list(range(301,324))

always_arc, normal_arc, congest_arc = [], [], []

for r in range(nbRoute):
    if (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in (normal_nodes+virtual_nodes)) and (EV_routes[r,1] != EV_routes[r,2]):
        normal_arc.append(r)
    elif (EV_routes[r,1] in (normal_nodes+virtual_nodes)) and (EV_routes[r,2] in congest_nodes):
        congest_arc.append(r)
    else:
        always_arc.append(r)


In [ ]:
# def solution_check(y_pred_binary, model, index_test, EV_schedules):

#     variables = model.getVars()
#     nbEV = int(EV_schedules.shape[0]/4)

#     # ========================================= initialise parameters ===================================================

#     EVArcStatus = np.zeros((nbScen, nbEV, nbRoute, nbTime-1))
#     EVArcStatusIndex = np.zeros((nbScen, nbEV, nbRoute, nbTime-1))
#     EVChargeStatus = np.zeros((nbScen, nbEV, nbCS, nbTime))
#     EVChargeStatusIndex = np.zeros((nbScen, nbEV, nbCS, nbTime))
#     EVDischargeStatus = np.zeros((nbScen, nbEV, nbCS, nbTime))
#     EVDischargeStatusIndex = np.zeros((nbScen, nbEV, nbCS, nbTime))

#     id_count = 0
#     for sc in range(nbScen):
#         for k in range(nbEV):
#             # for each EV get each attribute and order it based on EV number
#             for r in range(nbRoute):
#                 for t in range(nbTime-1):
#                     EVArcStatus[sc,k,r,t] = y_pred_binary[id_count]
#                     EVArcStatusIndex[sc,k,r,t] = y_pred_binary[id_count]
#                     id_count = id_count + 1

#             for c in range(nbCS):
#                 for t in range(nbTime):
#                     EVChargeStatus[sc,k,c,t] = y_pred_binary[id_count]
#                     EVChargeStatusIndex[sc,k,c,t] = y_pred_binary[id_count]
#                     id_count = id_count + 1

#             for c in range(nbCS):
#                 for t in range(nbTime):
#                     EVDischargeStatus[sc,k,c,t] = y_pred_binary[id_count]
#                     EVDischargeStatusIndex[sc,k,c,t] = y_pred_binary[id_count]
#                     id_count = id_count + 1

#     # ========================================= checking Arc constestts ===================================================
#     # set disable arc to 0 and check activated arc
#     for sc in range(nbScen):
#         for s in range(nbTime-1):
            
#             activate_arc = (always_arc + congest_arc) if traffic[s] else (always_arc + normal_arc)
#             disable_arc = normal_arc if traffic[s] else congest_arc
            
#             EVArcStatus[sc,:,np.array(disable_arc),s] = 0

#         # check for EVArcStatus sum == 1    
#         MoveStatus = np.sum(np.round(np.clip(EVArcStatus[sc], 0, None)), axis=1)
#         # print(MoveStatus)
#         for i in range(MoveStatus.shape[0]):
#             for j in range(MoveStatus.shape[1]):

#                 activate_arc = (always_arc + congest_arc) if traffic[j] else (always_arc + normal_arc)

#                 if MoveStatus[i,j] == 0:
#                     EVArcStatus[sc,i,activate_arc,j] = -1
#                 elif MoveStatus[i,j] > 1:
#                     EVArcStatus[sc,i,:,j] = -1
#                     # max = np.argmax(EVArcStatus[i,:,j])
#                     # EVArcStatus[i,:,j] = 0
#                         # EVArcStatus[i,max,j] = 1

#         MoveStatus = np.sum(np.round(np.clip(EVArcStatus[sc], 0, None)), axis=1)
#         # print(MoveStatus)

#         # check EVArc Continuity
#         for k in range(nbEV):
#             for s in range(nbTime-2):
#                 from_arc = np.max(np.round(EVArcStatus[sc,k,:,s]))
#                 to_arc = np.max(np.round(EVArcStatus[sc,k,:,s+1]))

#                 if from_arc == 1 and to_arc == 1:
#                     from_node = np.argmax(np.round(EVArcStatus[sc,k,:,s]))
#                     to_node = np.argmax(np.round(EVArcStatus[sc,k,:,s+1]))

#                     activate_arc = (always_arc + congest_arc) if traffic[s] else (always_arc + normal_arc)
                    
#                     if EV_routes[from_node, 2] != EV_routes[to_node, 1]:
#                         EVArcStatus[sc,k,:,s] = -1
#                         EVArcStatus[sc,k,:,s+1] = -1
#                 else:
#                     EVArcStatus[sc,k,:,s] = -1
#                     EVArcStatus[sc,k,:,s+1] = -1

#         # check schedule
#         nbSchedule = EV_schedules.shape[0]
#         for k in range(nbSchedule):
#             schedule = EV_schedules[k]  # [EV, destination, time]

#             route_index = (EV_routes[:,1]==schedule[1]).nonzero()[0] if schedule[2] == 0 else (EV_routes[:,2]==schedule[1]).nonzero()[0]
#             time = 0 if schedule[2] == 0 else schedule[2]-1

#             activate_arc = (always_arc + congest_arc) if traffic[time] else (always_arc + normal_arc)
#             disable_arc = normal_arc if traffic[time] else congest_arc

#             MoveStatus = np.sum(np.round(np.clip(EVArcStatus[sc,schedule[0], route_index, time], 0, None)))
#             if MoveStatus == 0:
#                 EVArcStatus[sc,schedule[0],activate_arc,time] = 0
#                 EVArcStatus[sc,schedule[0],route_index,time] = -1
#             elif MoveStatus > 1:
#                 EVArcStatus[sc,schedule[0],activate_arc,time] = -1
#                 # max = np.argmax(EVArcStatus[schedule[0],route_index,time])
#                 # EVArcStatus[schedule[0],:,time] = 0
#                 # EVArcStatus[schedule[0],max,time] = 1

#     # ========================================= checking charge constestts ===================================================
#     stationary_index = np.array((EV_routes[:,1] == EV_routes[:,2]).nonzero()[0])
#     charging_index = np.array([i for i, e in enumerate(EV_routes[:,1]) if e in charging_station])
#     charge_index = [i for i, e in enumerate(stationary_index) if e in charging_index]
#     for k in range(nbEV):
#         for i, ii in enumerate(stationary_index[charge_index]):
#             for s in range(nbTime-1):
#                 if not ((np.round(np.clip(EVChargeStatus[sc,k,i,s+1], 0, None)) + np.round(np.clip(EVDischargeStatus[sc,k,i,s+1], 0, None))) <= np.round(np.clip(EVArcStatus[sc,k,ii,s], 0, None))):
#                     EVChargeStatus[sc,k,i,s+1] = -1
#                     EVDischargeStatus[sc,k,i,s+1] = -1

#                     activate_arc = (always_arc + congest_arc) if traffic[s] else (always_arc + normal_arc)
#                     EVArcStatus[sc,k,:,s] = -1

#     ##### replacing y_predict ######
#     corrected_y = []

#     for sc in range(nbScen):
#         for k in range(nbEV):
#             # for each EV get each attribute and order it based on EV number
#             for r in range(nbRoute):
#                 for t in range(nbTime-1):
#                     corrected_y.append(EVArcStatus[sc,k,r,t])

#             for c in range(nbCS):
#                 for t in range(nbTime):
#                     corrected_y.append(EVChargeStatus[sc,k,c,t])

#             for c in range(nbCS):
#                 for t in range(nbTime):
#                     corrected_y.append(EVDischargeStatus[sc,k,c,t])

#     return corrected_y


In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

gurobi_env = gb.Env()
gurobi_env.setParam("OutputFlag", 0)

# loop through all test models and calculate average optimization time
opt_time_feas = []
opt_val_feas = []
opt_ones_feas = []
opt_utilized_feas = []
for i, x in enumerate(model_test):
    path = os.path.join(os.getcwd(), "dataGeneration/model_test")

    model = gb.read(os.path.join(path, f"coordination_{x}.mps"), env=gurobi_env)
    model.setParam("OutputFlag", 0)
    model.setParam("TimeLimit", 120*60)

    ################# timing the first code block #####################
    start = time.time()

    # deep learning prediciton
    # do a for loop to perform perdiction for each scenario
    output_append = torch.tensor([]).to(device)

    for s in range(nbScen):
        inputs = torch.tensor(np.expand_dims(test_dataset.X[i][s], axis=0), dtype=torch.float32) 
        inputs = inputs.to(device)    
        outputs = net(inputs) 

        output_append = torch.concat((output_append, outputs),dim=1)

    output_append = output_append.reshape(-1,)

    ################## end #########################

    y_pred_binary =  (output_append).reshape(-1,).cpu().detach().numpy()
    # y_pred_binary = y_test[i].flatten()
    nbEV = int(y_test[i].shape[1]/nbOut)
    print(nbEV)

    modelVars = model.getVars()

    one_threshold = (np.mean(mean_one))
    zero_threshold = (1-np.mean(mean_zero))
    # print(one_threshold, zero_threshold)

    bin_id = []
    ##### Build the index for the variables ######
    for sc in range(nbScen):
        for k in range(nbEV):
            # for each EV get each attribute and order it based on EV number
            for r in range(nbRoute):
                for t in range(nbTime-1):
                    var = model.getVarByName(f"EVArcStatus[{sc},{k},{r},{t}]")

                    bin_id.append(var.index)

            for c in range(nbCS):
                for t in range(nbTime):
                    var = model.getVarByName(f"EVChargeStatus[{sc},{k},{c},{t}]")

                    bin_id.append(var.index)
                    
            for c in range(nbCS):
                for t in range(nbTime):
                    var = model.getVarByName(f"EVDischargeStatus[{sc},{k},{c},{t}]")

                    bin_id.append(var.index)

    # use equality constestt from here on
    for j in range(len(y_pred_binary)):
        if (y_pred_binary[j] >= one_threshold or y_pred_binary[j] <= zero_threshold):
        # if (y_pred_binary[j] >= 0):

            modelVars[bin_id[j]].setAttr("LB", round(y_pred_binary[j]))
            modelVars[bin_id[j]].setAttr("UB", round(y_pred_binary[j]))
            
    end = time.time()
    runtime1 = end - start

    opt_ones_feas.append(np.count_nonzero(y_pred_binary >= one_threshold))
    opt_utilized_feas.append(np.count_nonzero(y_pred_binary >= one_threshold)+  np.count_nonzero(y_pred_binary <= zero_threshold))

    ################# timing the second code block #####################
    start = time.time()

    print("solving...")
    model.optimize()

    end = time.time()
    runtime2 = end - start
    runtime = runtime1 + runtime2
    ################## end #########################
    
    try:
        print("Optimization time for model ", i, ": ", model.ObjVal)
        print("Optimization value for model ", i, ": ", runtime)
        opt_time_feas.append(runtime)
        opt_val_feas.append(model.ObjVal) 

    except:
        print("infeasible, reoptimising")

        model.dispose()

        ###### reset model and re-fix the values
        model = gb.read(os.path.join(path, f"coordination_{x}.mps"), env=gurobi_env)
        model.setParam("OutputFlag", 0)
        model.setParam("TimeLimit", 120*60)

        ################# timing the code block #####################
        start = time.time()

        one_threshold = (np.mean(mean_one)) + 0.2
        zero_threshold = (1 - np.mean(mean_zero))

        modelVars = model.getVars()

        # use equality constestt from here on
        for j in range(len(y_pred_binary)):
            if (y_pred_binary[j] >= one_threshold or y_pred_binary[j] <= zero_threshold):
            # if (y_pred_binary[j] >= 0):

                modelVars[bin_id[j]].setAttr("LB", round(y_pred_binary[j]))
                modelVars[bin_id[j]].setAttr("UB", round(y_pred_binary[j]))
  
        end = time.time()
        runtime3 = end - start
        
        opt_ones_feas.append(np.count_nonzero(y_pred_binary >= one_threshold))
        opt_utilized_feas.append(np.count_nonzero(y_pred_binary >= one_threshold)+  np.count_nonzero(y_pred_binary <= zero_threshold))

        ################# timing the second code block #####################
        start = time.time()

        model.optimize()

        end = time.time()
        runtime4 = end - start
        runtime = runtime + runtime3 + runtime4 

        try:
            print("Optimization time for model ", i, ": ", model.ObjVal)
            print("Optimization value for model ", i, ": ", runtime)
            opt_time_feas.append(runtime)
            opt_val_feas.append(model.ObjVal) 
        except:
            print('infeasible')
            opt_time_feas.append(0)
            opt_val_feas.append(0)

            # model.computeIIS()

            # for c in model.getConstrs():
            #     if c.IISConstr: print(f'\t{c.constrname}: {model.getRow(c)} {c.Sense} {c.RHS}')

            # for v in model.getVars():
            #     if v.IISLB: print(f'\t{v.varname} ≥ {v.LB}')
            #     if v.IISUB: print(f'\t{v.varname} ≤ {v.UB}')

    
    model.dispose()



In [ ]:
opt_dict_feas = {
    "opt_time": opt_time_feas,
    "opt_val": opt_val_feas,
    "opt_ones": opt_ones_feas,
    "opt_utilized": opt_utilized_feas
}

result_path = os.path.join(os.getcwd(), f"Results")
with open(os.path.join(result_path, f"transformer_opt_feas_test_nopadding_{interval}.pkl"), 'wb') as f:
    pickle.dump(opt_dict_feas, f)

In [ ]:
with open(os.path.join(result_path, f"transformer_opt_feas_test_nopadding_{interval}.pkl"), 'rb') as f:
    opt_dict_feas = pickle.load(f)

opt_time_feas = opt_dict_feas["opt_time"]
opt_val_feas = opt_dict_feas["opt_val"]
opt_ones_feas = opt_dict_feas["opt_ones"]
opt_utilized_feas = opt_dict_feas["opt_utilized"]

In [ ]:
# flatten opt_time
opt_time_feas = np.array(opt_time_feas)
opt_val_feas = np.array(opt_val_feas)
opt_val = np.array(opt_val)

infeasible_samples = np.where(opt_time_feas == 0)[0]
feasible_sample = np.where(opt_time_feas != 0)[0]
print(infeasible_samples)
# print(feasible_sample)

opt_time_feas[infeasible_samples] = np.array(opt_time)[infeasible_samples]
opt_val_feas = opt_val_feas[feasible_sample]
opt_val = opt_val[feasible_sample]

opt_time_feas_mean = np.mean(opt_time_feas)
opt_val_feas_mean = np.mean(opt_val_feas)

print("Average optimization time: ", np.mean(opt_time_feas))
print("Average optimization value: ", np.mean(opt_val_feas))
print("Number of feasible model: ", len(opt_val_feas))
# print("Number of ones: ", opt_ones_feas)
# print("Number of variable utilized ", opt_utilized_feas)

### Calculate time sped up by and optimality difference

In [ ]:
speedup_time = 1 - (np.array(opt_time_feas)/np.array(opt_time))
obj_loss = ((np.array(opt_val_feas)-np.array(opt_val))/np.array(opt_val))
feasible_model_feas = (len(opt_time_feas) - len(infeasible_samples))/ len(model_test) * 100


print(f"Time sped up: {speedup_time*100} %")
print(f"Optimality Loss: {obj_loss*100} %")
print(f"Feasible model: {feasible_model_feas} %")

